# このスライドへのアクセス方法

1. 「nbviewer.jupyter.org」というサイトへアクセス
2. 「mrsekut」で検索
3. 「RealWorldHTTP > myproduct > slide > HTTP2.ipynb」
4. 右上のプレゼントマークでスライドっぽく表示
5. スペースキーで順番に移動

[リンク](http://nbviewer.jupyter.org/github/mrsekut/RealWorldHTTP/blob/master/myproduct/slide/HTTP2.ipynb)


# HTTP/2.0について

丸末皓太

# 自己紹介

### 丸末皓太
- 大阪府立大学(いちおうたぶん2回生)
- 休学中
- 府大生向けの「Opulency」というアプリを作った
- 現在はtryangle社でばりばりアプリ開発してる卍卍
- 最近興味あること
    - Typescript: ReactNative
    - Haskell: 関数型プログラミング
    - Go: ネットワーク, プログラミング言語そのもの
    - Rust: OSなどの低レイヤ
    - テスト  



- 大学に入ってから本格的にプログラミングを始めた
- 1回生でprogateなどでhtml,cssとか、普通にvimとか触った
- 2回生になって機械学習の勉強を始めたり、reactでopulencyというアプリを作り始めた
- opulencyは今年の4月にリリースして2週間で500人くらいにいれてもらったが、リリースした瞬間にモチベが切れ、終わった

![mrsekut](https://github.com/mrsekut/myStorage/blob/master/img/mrsekut_circle.png?raw=true)

### SNS
- blog: [mrsekutの備忘録: http://mrsekut.site/](http://mrsekut.site/)
- twitter: [@mrsekut](https://twitter.com/mrsekut)
- github: [mrsekut](https://github.com/mrsekut)

# 発表のながれ
★あとで修正
1. 発表の目的
1. 従来のHTTP
1. HTTP/2の概要
1. 試してみる
1. なんか
1. なんか


# 発表の目的

- 浅く広く紹介します
- キーワードを拾っていってもらえばと思います

# 従来のHTTP


## 概要

- リクエストとレスポンス
- HTTP/1.1
    - 1997年に策定

## HTTP1.1の課題

- リクエストの多重化
    - ページが大きくなり必要なリソースが増えた
    - 1RTTで1リクエスト/1レスポンスしかできない
- パイプライン化
    - 同時に複数のリクエストを送れるようになった
    - しかし、同じ順序でレスポンスしないといけない
    - HoL(Head of Line)ブロッキング問題
    - ![wikiから引用](https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/HTTP_pipelining2.svg/640px-HTTP_pipelining2.svg.png)
      



- パイプライン化
    - 前回のリクエストの完了を待たずに次のリクエストを送れるようになった
    - 
    - 一つ処理に時間がかかるものがあると、後続のレスポンスは全てブロックされる
    - これをHoLブロッキング問題と呼ぶ

# HTTP/2の概要

## 誕生の背景

- アクセスの増加
- HTTP/1.1から16年ぶりの改訂
- GoogleのSPDYから生まれた
- 当初のプロジェクトの目標はページ読み込み時間の50%減

- LTEなどの普及によりエンドユーザーの通信速度は上昇したが、サーバー側のページロードの速度はそれに比例せず1.6Mbpsほどで頭打ちしていた
- 2009年からgoogleが独自にweb高速化の取り組みの一環からSPDYというプロトコルを作っていた
    - Twitter,Facebookでも使われるようになった
    - HTTP/2はこのSPDYを改良したもの

## HTTP/2の特徴

- ヘッダの圧縮
- サーバープッシュ
- リクエストの優先度制御
- バイナリプロトコル
- シングルコネクション???


### ヘッダの圧縮


- ヘッダー圧縮
    - 従来の圧縮に加え、ヘッダ部分の圧縮ができる
    - 静的テーブル、動的テーブル
        - 使用頻度の高いヘッダ、一度送信したヘッダなどに番号を振り、次からそれを使う
    - HPACK
        - 動的に辞書を更新し、2回目移行はインデックスを用いる
        - 文字列をHuffman Encodingによって圧縮
            - 高頻度で用いられる文字を少ないビット数で表現する
            - バイナリ形式
            - [超わかりやすい解説記事](https://qiita.com/iwanaga/items/98f60003c0114e04095e)
            - [超詳しい解説記事](http://syucream.github.io/hpack-spec-ja/rfc7541-ja.html)
        - 差分のみを送信する
- https://qiita.com/0xfffffff7/items/39b944e3845ab3776b63

- HTTP/1.1のヘッダは大きい
    - リクエストは最低でも300Byte程度
    - Cookieなどで更に増える
    
- HTTP/1.1ではbodyをエンコードするのにgzipでサイズを圧縮する機構がある
    - CRIMEという攻撃手法に備えるためヘッダの圧縮には使えない

- 膨大なHTTPリクエスト/レスポンスのログのヘッダの文字の出現頻度を集計し、ハフマンツリーを作成
- 高頻度で使われる文字列とは母音とか
    - たとえばaとかが普通にすると8bit必要だったけど、例えば01みたいな2bitで耐える
- 「\(バックスラッシュ)」とかあまり使われない文字は19bitとか必要になり、逆に増える

- この「ヘッダ圧縮」だけ取り上げて深めてみても面白そうなトピックだなと感じた

- ちなみに「hpack」で検索するとhaskellのツールがでてくるが　、コレは関係ない
    - jsでいうpackage.json的なファイルからコンパイルしたりするファイルを作成するやつ
    
    
    

### 多重化

- 1つの接続上にストリームという仮想的な双方向シーケンスを作り多重化
- 同時に100以上のリクエストを送信可能
- 並列に扱うことができる
    - レスポンスの順序に制限なし
    - HoLブロッキング問題を解決
- データにはフレームという単位にわけられ、ストリーム上でやりとりされる
    - 0-9の10種類がありIDが振られている
    - 例えば
        - ボディ(0),ヘッダ(1)
        - ストリームの優先順位を指定(2)
        - 接続の生存状態を調べる(7)
    - クライアント側は奇数、サーバ側は偶数番号を割り当て、重複を防いでいる
    - ストリームはサーバ側でもクライアント側でも開始可能


- 「フレーム」というのはHTTP/2の通信の最小単位
    - それぞれにフレームヘッダが含まれる

### リクエストの優先度制御

- クライアントがサーバーにレスポンス優先度を指定できる
- 優先度を付けることでユーザ体感速度を大幅に向上させることができる
- 重み付け
    - 値に比例したバンド幅の配分★
- 依存関係
    - 依存しているレスポンス先に送信
    - グループ化にも利用
    
##### 参考
39~
https://www.slideshare.net/kazuho/http-58452175

### サーバープッシュ

- サーバーがクライアントの発行するリクエストを予測してレスポンスを返す
- 0RTTにすることができる



- 「重み付け」と「依存関係」の2種類の制御手法の組み合わせで優先度を制御している
- サーバーは指定された優先度を「参考にする」★

### バイナリベース


  - サーバーがパースを簡単に行えるようになり、よりコンパクトに、そしてエラーを減らすことができる
  - 脆弱性を防ぐ
    - HTTP Response Splitting Attack
    - レスポンス分割攻撃
  - 転送データ量の低減
  - 全てのデータは「フレーム」に分解して送受信


### シングルコネクション

  - ウェブサイトを読み込むためにサーバーに接続する回数はたったの一度
  - TCPコネクションを張るために必要なラウンドトリップの回数を減らす




# 試してみる

## HTTPプロトコルバージョンを確認する

httpのgoogle

In [1]:
!go run ../https_server/http_google.go

Protocol Version: HTTP/1.1


httpsのgoogle

In [ ]:
!go run ../https_server/https_google.go

## chromeで確認する方法


# 対応

# さいごに

- もうちょい詳細にしてブログにまとめます

# 参考文献

- Real World HTTP
- https://ja.wikipedia.org/wiki/HTTP/2
- https://blog.redbox.ne.jp/http2-cdn.html
- http://www.seojapan.com/blog/everyone-moving-http2
- https://www.slideshare.net/kazuho/http-58452175
- https://developers.google.com/web/fundamentals/performance/http2/?hl=ja